In [7]:
%config Completer.use_jedi = False

import pandas as pd
import os
import subprocess
from video import Video
import random
from extractors import ExtractorFactory
from clusters import ClusterFactory
from mxnet import nd, gpu
import io
import cv2
import finch
from pathlib import Path
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
import numpy as np
import math
from pprint import pprint
from accuracy import Accuracy
from finch import FINCH
from npy import Npy

### INRIA Utils

In [8]:
import xmltodict
from pprint import pprint 


videos_num_frames = {}
for tp in ['changing_tire', 'coffee', 'jump_car', 'repot', 'cpr']:
    with open(f'./data_new/data_release/{tp}.txt', 'r') as f:
        for ele in f:
            line = ele.split('\n')[0]
            split_line = line.split("\t")
            videos_num_frames[split_line[1]] = int(split_line[-2])

def map_seglabels_to_framelabels(labels, video_len, seg_len=32):
    """ Map segments(32 frames pieces of a video) labels to a framewise label """
    frames_labels = []

    for seg_label in labels:
        frames_labels = frames_labels + [seg_label] * seg_len
    return np.array(frames_labels[:video_len])

def read_gt(path):
    split_path = path.split('/')
    base_path = split_path[:3]
    fname = split_path[-1].split('.')[0]
    
    gt_fname = fname + '.xgtf'
    gt_path = os.path.join(*base_path, 'annot', gt_fname)
    
    with open(gt_path) as f:
        doc = xmltodict.parse(f.read())
    annot = doc['viper']['data']['sourcefile']
    
    try:
#         int(annot['file']['attribute'][1]['data:dvalue']['@value'])
        return annot['object'], videos_num_frames[fname]
    except Exception as e:
        pprint(annot)
        raise e

def gt_to_array(path):
    annot, num_frames = read_gt(path)
    frames_labels = []
    last_end = 0
    bg_segs = []
    gt_segs = []
    
    label_set = set([gt_set['@name'] for i, gt_set in enumerate(annot)])
    labels = {a: i
              for i, a in enumerate(label_set)}

    for i, gt_seg in enumerate(annot):
        spans = gt_seg['@framespan'].split(' ')
        if len(spans) == 1:
            start, end = gt_seg['@framespan'].split(':')
            gt_segs.append({'@framespan': (int(start), int(end)), 'label': labels[gt_seg['@name']]})
        else:
            for span in spans:
                start, end = span.split(':')
                gt_segs.append({'@framespan': (int(start), int(end)), 'label': labels[gt_seg['@name']]})
    
    gt_segs = sorted(gt_segs, key=lambda k: k['@framespan'][0])

    for i, gt_seg in enumerate(gt_segs):
        start, end = gt_seg['@framespan']   
        i = gt_seg['label']
        if last_end != start:
            if last_end == 0:
                bg_segs.append((last_end, start - 1))
                span = start - last_end
            else:
                bg_segs.append((last_end + 1, start - 1))
                span = start - last_end - 1
            frames_labels = frames_labels + [-1] * span
            
        frames_labels = frames_labels + [i] * (end - start + 1)
        last_end = end
    
    if len(frames_labels) != num_frames:
        span = num_frames - last_end
        frames_labels = frames_labels + [-1] * span
    
    frames_labels = frames_labels[:num_frames]
    
    if len(frames_labels) != num_frames:
        print(len(frames_labels))
        print(num_frames)
        print(annot)
        raise
    
    return np.array(frames_labels), bg_segs

def positional_encoding(data: np.ndarray) -> np.ndarray:
    def get_sinusoid_encoding_table(length, d_model):

        def cal_angle(position, hid_idx):
            return position / np.power(10000, 2 * (hid_idx // 2) / d_model)

        def get_posi_angle_vec(position):
            return [cal_angle(position, hid_j) for hid_j in range(d_model)]

        sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(length)])

        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

        return sinusoid_table

    d_model = data.shape[1]
    length = data.shape[0]

    pe = get_sinusoid_encoding_table(length, d_model)

    return data + pe
    
gt, bg_segs = gt_to_array(paths[0])

NameError: name 'paths' is not defined

In [9]:
from sklearn.metrics import f1_score

class EvalSegmentation:
    
    def __init__(self):
        self.mofs = {}
        self.ious = {}
        self.f1_scores = {}
        
    def setup_key(self, key):
        if key not in self.mofs:
            self.mofs[key] = []
        
        if key not in self.ious:
            self.ious[key] = []
            
        if key not in self.f1_scores:
            self.f1_scores[key] = []
            
    def __call__(self, predicted, ground_truth, key):
        acc = Accuracy()
        acc.predicted_labels = predicted
        acc.gt_labels = ground_truth
        acc.mof()
        acc.mof_classes()
        acc.iou_classes()
        stats = acc.stat()['iou']
        iou = stats[0] / stats[1]
        self.ious[key].append(iou)
        self.mofs[key].append(acc.mof_val())
        
        mapped_pred = self.map_pred2gt(ground_truth, predicted)
        self.f1_scores[key].append(f1_score(ground_truth, mapped_pred, average='micro'))
    
    def map_pred2gt(self, gt, pred):
        gt2pred = self.get_gt2preds_mapping(gt, pred)
        mapped_pred = np.copy(pred)
        for gt_label in np.unique(gt):
            pred_label = gt2pred.get(gt_label, None)
            mapped_pred[mapped_pred == pred_label] = gt_label
        return mapped_pred
        
    def get_gt2preds_mapping(self, gt, pred):
        acc = Accuracy()
        acc.predicted_labels = pred
        acc.gt_labels = gt
        return acc.get_gt2labels_mapping()
            
    def report_results(self):
        def calculate(metrics_per_key, metric_name):
            print(metric_name)
            acc_metric = []  # accumulated metric
            for key, metrics_per_sample in metrics_per_key.items():
                print(key, np.mean(metrics_per_sample))
                acc_metric.append(np.mean(metrics_per_sample))
            print(f"Final {metric_name}: {np.mean(acc_metric)}")
        calculate(self.ious, 'IoU')
        calculate(self.mofs, 'MoF')
        calculate(self.f1_scores, 'F1-score')

evalseg = EvalSegmentation()

In [11]:
from dataclasses import dataclass, field
from typing import List, Tuple
from clusters import ClusterStrategy

@dataclass
class SegmentationInria:
    with_pe: bool = None
    cluster: ClusterStrategy = None
    paths: List[str] = field(default_factory=lambda: get_video_paths(root_folder='data_new', video_ext='avi'))
    features_seg_size: int = None
    eval_seg: EvalSegmentation = EvalSegmentation()
        
    def get_features(self, path: str) -> np.ndarray:
        pass
    
    def remove_bg(self, pred_labels, gt) -> Tuple[np.ndarray, np.ndarray]:
        bg_idx = np.where(gt == -1)[0]
        bg_idx_rem = bg_idx[:int(len(bg_idx) * .75)]
        pred_labels = np.delete(pred_labels, bg_idx_rem, axis=0)
        gt = np.delete(gt, bg_idx_rem, axis=0)
        return pred_labels, gt
    
    def segment_a_video(self, path):
        gt, bg_segs = gt_to_array(path)
        features = self.get_features(path)
        pred_labels = self.cluster.auto(features)
        pred_labels = map_seglabels_to_framelabels(pred_labels, gt.shape[0], seg_len=self.features_seg_size)

        if gt.shape[0] > pred_labels.shape[0]:
            gt = gt[:pred_labels.shape[0]]
        elif gt.shape[0] < pred_labels.shape[0]:
            pred_labels = pred_labels[:gt.shape[0]]

        pred_labels, gt = self.remove_bg(pred_labels, gt)
                
        activity = path.split("/")[2]
        mapping = self.eval_seg.get_gt2preds_mapping(gt, pred_labels)
        self.eval_seg.setup_key(activity)
        self.eval_seg(pred_labels, gt, activity)
        return mapping, gt, pred_labels
    
    def segment(self):
        for i, path in enumerate(self.paths):
            try:
                gt, bg_segs = gt_to_array(path)
                features = self.get_features(path)
                pred_labels = self.cluster.auto(features)
                pred_labels = map_seglabels_to_framelabels(pred_labels, gt.shape[0], seg_len=self.features_seg_size)

                if gt.shape[0] > pred_labels.shape[0]:
                    gt = gt[:pred_labels.shape[0]]
                elif gt.shape[0] < pred_labels.shape[0]:
                    pred_labels = pred_labels[:gt.shape[0]]

                pred_labels, gt = self.remove_bg(pred_labels, gt)
                
                activity = path.split("/")[2]
                self.eval_seg.setup_key(activity)
                self.eval_seg(pred_labels, gt, activity)

            except Exception as e:
                print(path)
                print(e)
                raise e
    
@dataclass
class SegSlowFast(SegmentationInria):
    
    features_seg_size: int = 32
    
    def get_features(self, path: str) -> np.ndarray:
        features_path = "_".join([ExtractorFactory.SLOWFAST.value, str(self.features_seg_size)])
        video = Video(path, features_path)
        features = video.features(with_pe=self.with_pe)
        return features

@dataclass
class SegI3D(SegmentationInria):
    
    features_seg_size: int = 10

    def get_i3d_feature_path(self, video_path: str) -> Tuple[str, str]:
        features_base_path = f'./inria_features_{self.features_seg_size}frames/i3d'
        video_fname = video_path.split('/')[-1]
        fname = video_fname.split('.')[0]
        rgb_feat = "_".join([fname, 'rgb']) + '.npy'
        flow_feat = "_".join([fname, 'flow']) + '.npy'

        rgb_path = os.path.join(features_base_path, rgb_feat)
        flow_path = os.path.join(features_base_path, flow_feat)
        return rgb_path, flow_path

    def get_features(self, path: str) -> np.ndarray:
        rgb, flow = self.get_i3d_feature_path(path)
        features = np.concatenate([np.load(flow), np.load(rgb)], axis=1)
        last_frames = np.array([features[-1] for _ in range(10)])
        features = np.concatenate([features, last_frames], axis=0)
        if self.with_pe:
            return positional_encoding(features)
        else:
            return features

@dataclass
class SegDino(SegmentationInria):
    
    features_seg_size: int = 32
    
    def get_features(self, path: str) -> np.ndarray:
        features_path = "_".join([ExtractorFactory.SLOWFAST.value, str(1)])
        video = Video(path, features_path)
        features = video.features(with_pe=self.with_pe, avg=True, n_rows=self.features_seg_size)
        return features

### Dino

In [ ]:
seg = SegDino(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=64)
seg.segment()
seg.eval_seg.report_results()

## I3D

### 10 Frames window

### FINCH + PE

In [15]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.1428592861283747
coffee 0.15677779009143825
jump_car 0.2410226116961634
repot 0.1877462846482402
cpr 0.23950889796100594
Final IoU: 0.1935829741050445
MoF
changing_tire 0.34260689968317454
coffee 0.46259859287434973
jump_car 0.6195635161019869
repot 0.4716239547388482
cpr 0.46605864665043895
Final MoF: 0.47249032200975966
F1-score
changing_tire 0.2746638044097363
coffee 0.4222502026640185
jump_car 0.6030957545168408
repot 0.42292823634834753
cpr 0.4381697227123791
Final F1-score: 0.4322215441302644


### FINCH

In [16]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.13503857117041979
coffee 0.20253044169484283
jump_car 0.20383611977840432
repot 0.19673634447032032
cpr 0.26976946162594023
Final IoU: 0.20158218774798548
MoF
changing_tire 0.3654047711530034
coffee 0.5238425388581721
jump_car 0.5827653353431912
repot 0.4982853953739562
cpr 0.5223081162650306
Final MoF: 0.49852123139867077
F1-score
changing_tire 0.26795840032332274
coffee 0.44412421524551554
jump_car 0.561277079433804
repot 0.44051545301117406
cpr 0.45712662681317595
Final F1-score: 0.43420035496539844


### KMeans + SS + PE

In [19]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("KMeans")(n=2))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.21093392390129984
coffee 0.2810115283540194
jump_car 0.3152120332173841
repot 0.25634344383376645
cpr 0.2806910435791001
Final IoU: 0.26883839457711395
MoF
changing_tire 0.3356331175994345
coffee 0.4287682544979242
jump_car 0.49065349252028206
repot 0.40230438949361813
cpr 0.4038886020089967
Final MoF: 0.4122495712240511
F1-score
changing_tire 0.26207157499809663
coffee 0.37472453600953903
jump_car 0.47137022104364434
repot 0.3655696847846977
cpr 0.3624671679696988
Final F1-score: 0.3672406369611353


### KMeans + SS

In [20]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("KMeans")(n=2))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.21611739346166264
coffee 0.31703159143020326
jump_car 0.30614306930482205
repot 0.27241191157345523
cpr 0.31204870437557436
Final IoU: 0.2847505340291435
MoF
changing_tire 0.3663653571030707
coffee 0.48354670573335173
jump_car 0.49560333858803296
repot 0.44026940468962034
cpr 0.448833569507202
Final MoF: 0.44692367512425557
F1-score
changing_tire 0.275124617365379
coffee 0.4159325526273011
jump_car 0.4683961166714614
repot 0.3980009891660877
cpr 0.40413420858417226
Final F1-score: 0.39231769688288026


### 16 frames window size

### FINCH

In [30]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=16)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.14030642827124676
coffee 0.19774276093862075
jump_car 0.20942009933333774
repot 0.19757362970060496
cpr 0.32117551031001107
Final IoU: 0.21324368571076424
MoF
changing_tire 0.35742982884693375
coffee 0.5019001716246072
jump_car 0.6218013881457297
repot 0.48548217416906875
cpr 0.4954311723257077
Final MoF: 0.49240894702240945
F1-score
changing_tire 0.2690283291394101
coffee 0.4558537678435428
jump_car 0.5992937913633433
repot 0.4441301255844918
cpr 0.45966208481937454
Final F1-score: 0.4455936197500325


In [31]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=16)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.15451042117186825
coffee 0.22703205497429746
jump_car 0.1791808762798116
repot 0.20819601093217993
cpr 0.28593331451104126
Final IoU: 0.21097053557383968
MoF
changing_tire 0.393142374212136
coffee 0.5382973203040232
jump_car 0.5822420519369192
repot 0.5137293556023992
cpr 0.5182093024320655
Final MoF: 0.5091240808975087
F1-score
changing_tire 0.28499747629435623
coffee 0.46239887476938396
jump_car 0.5584916466459062
repot 0.43947467271600354
cpr 0.452492016762511
Final F1-score: 0.4395709374376322


### KMEANS

In [33]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("KMeans")(n=2), features_seg_size=16)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.23571839201696995
coffee 0.31983383442842006
jump_car 0.2885852977753377
repot 0.3028235651957579
cpr 0.3329674260094023
Final IoU: 0.2959857030851776
MoF
changing_tire 0.38623564170182073
coffee 0.4977545003480842
jump_car 0.5107688079838034
repot 0.4597590133255721
cpr 0.4545846744436612
Final MoF: 0.46182052756058833
F1-score
changing_tire 0.30028011444042485
coffee 0.4366632224255589
jump_car 0.4836897958177323
repot 0.4068606722339177
cpr 0.40547552203357784
Final F1-score: 0.4065938653902423


In [32]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("KMeans")(n=2), features_seg_size=16)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.1815479863916414
coffee 0.26833595867539467
jump_car 0.20241647590977482
repot 0.2583796080180404
cpr 0.32212448502315266
Final IoU: 0.2465609028036008
MoF
changing_tire 0.41673489269006125
coffee 0.5689267833707407
jump_car 0.5601912820791706
repot 0.5248794196901037
cpr 0.532383251556799
Final MoF: 0.520623125877375
F1-score
changing_tire 0.3079123722207266
coffee 0.48911675498734153
jump_car 0.5301361873944381
repot 0.452897361633851
cpr 0.46515730195505833
Final F1-score: 0.4490439956382831


### I3D 24 Frames

In [36]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=24)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.22269267598032505
coffee 0.29403192493771657
jump_car 0.27351306351597887
repot 0.2837597199932912
cpr 0.32202168336586967
Final IoU: 0.2792038135586362
MoF
changing_tire 0.38846706109548224
coffee 0.500690955555201
jump_car 0.530071378447344
repot 0.46389399141099036
cpr 0.4550967438434875
Final MoF: 0.467644026070501
F1-score
changing_tire 0.3044445629253538
coffee 0.442862855428481
jump_car 0.5038499212822442
repot 0.41092802063646994
cpr 0.4129645666514313
Final F1-score: 0.41500998538479605


In [38]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=24)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.20258123876347292
coffee 0.27969062840236936
jump_car 0.23652904369554106
repot 0.26773055191025685
cpr 0.3001102983444363
Final IoU: 0.2573283522232153
MoF
changing_tire 0.3974224350387169
coffee 0.5190862838945497
jump_car 0.5315562301287832
repot 0.48746431012348795
cpr 0.4801898376665951
Final MoF: 0.48314381937042655
F1-score
changing_tire 0.2972556263044784
coffee 0.44501929217710123
jump_car 0.503969209063336
repot 0.4217575302132316
cpr 0.42709090455448634
Final F1-score: 0.41901851246252664


### I3D 32 Frames

In [52]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=32)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.19832657487937322
coffee 0.26831938942291805
jump_car 0.23315085971270258
repot 0.25665526256116195
cpr 0.30174061836669336
Final IoU: 0.25163854098856986
MoF
changing_tire 0.39587128926175935
coffee 0.5189169120040698
jump_car 0.5412216567954566
repot 0.4869754298586439
cpr 0.4816090313702891
Final MoF: 0.4849188638580437
F1-score
changing_tire 0.2964181719600475
coffee 0.44411279419870753
jump_car 0.5137769263708568
repot 0.42016073025239586
cpr 0.42920840952708705
Final F1-score: 0.42073540646181895


In [7]:
index_min = min(range(len(values)), key=values.__getitem__)seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=40)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.17805529668302
coffee 0.22771685028878333
jump_car 0.19331741582075038
repot 0.2240240477863761
cpr 0.3355783642061675
Final IoU: 0.23173839495701945
MoF
changing_tire 0.4015194358104095
coffee 0.529398684379253
jump_car 0.5955252233669125
repot 0.4947537024743304
cpr 0.5175151758557958
Final MoF: 0.5077424443773402
F1-score
changing_tire 0.3134002197271179
coffee 0.4547614991727481
jump_car 0.5608767688792926
repot 0.4583821147378034
cpr 0.4396378447084788
Final F1-score: 0.44541168944508813


In [11]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=40)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.1532321529036785
coffee 0.24125023122885933
jump_car 0.13708991744351798
repot 0.269347946147166
cpr 0.2834007058040955
Final IoU: 0.21686419070546345
MoF
changing_tire 0.420658228654649
coffee 0.5721159137286146
jump_car 0.5086497952071571
repot 0.5734624792142426
cpr 0.5685608891641775
Final MoF: 0.5286894611937683
F1-score
changing_tire 0.3089989519958658
coffee 0.4366788522798364
jump_car 0.48057740905976515
repot 0.5090303690678927
cpr 0.4859971744497786
Final F1-score: 0.4442565513706277


In [9]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=48)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.19658702988043616
coffee 0.2543551918523486
jump_car 0.22449978869023396
repot 0.24507933975583007
cpr 0.35417634691686933
Final IoU: 0.25493953941914366
MoF
changing_tire 0.4223981885803382
coffee 0.5586936313297919
jump_car 0.6081191796262686
repot 0.5169928720139609
cpr 0.5194180136923116
Final MoF: 0.5251243770485343
F1-score
changing_tire 0.33902437079705877
coffee 0.47319118328026805
jump_car 0.5650350524181565
repot 0.4816838950342356
cpr 0.43089687458422327
Final F1-score: 0.45796627522278843


In [12]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=48)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.16689163568816295
coffee 0.2466981073915139
jump_car 0.1541482419386781
repot 0.2424811958394466
cpr 0.28209275701126957
Final IoU: 0.21846238757381425
MoF
changing_tire 0.4275932169186173
coffee 0.5838516665852228
jump_car 0.5014054652466845
repot 0.566572120317934
cpr 0.5747442591606375
Final MoF: 0.5308333456458193
F1-score
changing_tire 0.3212678532077281
coffee 0.45927255846908266
jump_car 0.468776234704479
repot 0.4914974185458245
cpr 0.47791439718581913
Final F1-score: 0.44374569242258666


In [10]:
seg = SegI3D(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=64)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.20490772656762743
coffee 0.2842315420247562
jump_car 0.22884099329312438
repot 0.2666746285680824
cpr 0.36338513481966406
Final IoU: 0.2696080050546509
MoF
changing_tire 0.43673029690492576
coffee 0.5732025269179202
jump_car 0.5944532317571354
repot 0.5400827699119723
cpr 0.5232212709119289
Final MoF: 0.5335380192807765
F1-score
changing_tire 0.3537313412760951
coffee 0.4735825912598074
jump_car 0.5420209454640179
repot 0.49119557989462104
cpr 0.43829234820149104
Final F1-score: 0.4597645612192065


In [13]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=64)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.1578117124374305
coffee 0.2340855444457093
jump_car 0.15503621236419732
repot 0.25125315703840667
cpr 0.2782980570606406
Final IoU: 0.21529693666927688
MoF
changing_tire 0.4166217329733891
coffee 0.5797658187825401
jump_car 0.497789459789956
repot 0.5687124031733863
cpr 0.5741778741744089
Final MoF: 0.5274134577787362
F1-score
changing_tire 0.32000737403882723
coffee 0.4505286449771191
jump_car 0.4657015660867974
repot 0.509458700304623
cpr 0.4827485411134867
Final F1-score: 0.44568896530417074


In [53]:
seg = SegI3D(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'), features_seg_size=32)
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.1933401776383615
coffee 0.26691605348145114
jump_car 0.22120592252064122
repot 0.25311334344348374
cpr 0.29729013051925
Final IoU: 0.24637312552063753
MoF
changing_tire 0.3973185504152024
coffee 0.5272549857465189
jump_car 0.5368064997324419
repot 0.4939554743379694
cpr 0.48895114350629126
Final MoF: 0.48885733074768484
F1-score
changing_tire 0.293695581848309
coffee 0.45079398393614695
jump_car 0.5093493083120911
repot 0.42388894528895565
cpr 0.4359409230171649
Final F1-score: 0.4227337484805336


## Slowfast

### FINCH + PE

#### 32

In [ ]:
seg = SegSlowFast(with_pe=True, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

#### 40

In [143]:
seg = SegSlowFast(with_pe=True, features_seg_size=40, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.22834494457228383
coffee 0.3440988634153554
jump_car 0.25542658431515536
repot 0.3505823608636049
cpr 0.36346178496262066
Final IoU: 0.30838290762580406
MoF
changing_tire 0.452971989301585
coffee 0.5749322603084855
jump_car 0.5798952422748483
repot 0.542563105114701
cpr 0.5029109459829733
Final MoF: 0.5306547085965186
F1-score
changing_tire 0.34330882979650224
coffee 0.4713119647548142
jump_car 0.5350475773608971
repot 0.41783686826051775
cpr 0.3938979266474984
Final F1-score: 0.43228063336404593


#### 48

In [152]:
seg = SegSlowFast(with_pe=True, features_seg_size=48, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.2288098460669525
coffee 0.3649434107197503
jump_car 0.26008957899009744
repot 0.3688717165205745
cpr 0.38045196494135664
Final IoU: 0.3206333034477463
MoF
changing_tire 0.44973919067363777
coffee 0.5688369606382164
jump_car 0.5755367701729764
repot 0.5372306856834747
cpr 0.4931056768426267
Final MoF: 0.5248898568021864
F1-score
changing_tire 0.33953759793399996
coffee 0.46679500738352325
jump_car 0.5332581252309395
repot 0.4249768353027257
cpr 0.374156758366165
Final F1-score: 0.42774486484347074


#### 64

In [158]:
seg = SegSlowFast(with_pe=True, features_seg_size=64, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.24204992692153113
coffee 0.3720419682174637
jump_car 0.25426302477589474
repot 0.37047250930489634
cpr 0.38230222276065684
Final IoU: 0.32422593039608855
MoF
changing_tire 0.46118737200492677
coffee 0.575752964496237
jump_car 0.5690493720355989
repot 0.5383723589619895
cpr 0.4993811018356557
Final MoF: 0.5287486338668816
F1-score
changing_tire 0.34615674269250457
coffee 0.46389629177746167
jump_car 0.5280971465028826
repot 0.42529366682862274
cpr 0.3806961301155124
Final F1-score: 0.4288279955833968


In [37]:
seg = SegSlowFast(with_pe=True, features_seg_size=128, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.20902204597557436
coffee 0.29955616109170774
jump_car 0.1984651808179539
repot 0.2990732901702938
cpr 0.31372659543988024
Final IoU: 0.26396865469908204
MoF
changing_tire 0.4766596798306614
coffee 0.6002467467673575
jump_car 0.6106547950869842
repot 0.580359848882048
cpr 0.5071970336465881
Final MoF: 0.5550236208427277
F1-score
changing_tire 0.35267899778639294
coffee 0.46923788423698937
jump_car 0.5659220471277456
repot 0.42762788661705414
cpr 0.37696696391015444
Final F1-score: 0.43848675593566727


### FINCH

In [24]:
seg = SegSlowFast(with_pe=False, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.2402510343540955
coffee 0.33368257152291686
jump_car 0.3149711915462714
repot 0.31349391704029916
cpr 0.34023357075946703
Final IoU: 0.30852645704461
MoF
changing_tire 0.3819688783065075
coffee 0.4911199940411828
jump_car 0.4953979691791683
repot 0.460817077357683
cpr 0.4625049625333279
Final MoF: 0.4583617762835739
F1-score
changing_tire 0.2966669950191658
coffee 0.418705893842721
jump_car 0.4639398933717257
repot 0.41519284749569985
cpr 0.4193368859421304
Final F1-score: 0.4027685031342886


In [22]:
seg = SegSlowFast(with_pe=False, features_seg_size=40, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.19493916110332135
coffee 0.2922885927750841
jump_car 0.2267760229754518
repot 0.2934097467674206
cpr 0.33583696595300216
Final IoU: 0.268650097914856
MoF
changing_tire 0.4014361040595931
coffee 0.5404296021959444
jump_car 0.5211187833284432
repot 0.504446239515564
cpr 0.5095329848431167
Final MoF: 0.4953927427885323
F1-score
changing_tire 0.34425580228568087
coffee 0.48739564928629014
jump_car 0.4802113619487574
repot 0.4404757659926028
cpr 0.4344986374853702
Final F1-score: 0.43736744339974026


In [11]:
seg = SegSlowFast(with_pe=False, features_seg_size=48, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.209243967169658
coffee 0.32480293152955514
jump_car 0.20741919013868898
repot 0.3236523792682979
cpr 0.3503851731967915
Final IoU: 0.2831007282605983
MoF
changing_tire 0.42634699928935815
coffee 0.5675025529142145
jump_car 0.5444899764432034
repot 0.5266773294386549
cpr 0.5018941143580418
Final MoF: 0.5133821944886946
F1-score
changing_tire 0.3299590037954943
coffee 0.5258161394836945
jump_car 0.5051198000537558
repot 0.45331184881395287
cpr 0.39458938803120797
Final F1-score: 0.4417592360356212


In [12]:
seg = SegSlowFast(with_pe=False, features_seg_size=64, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.18354550056962843
coffee 0.29103970488983044
jump_car 0.19209100183564953
repot 0.28940850377581256
cpr 0.3230781804788175
Final IoU: 0.25583257830994766
MoF
changing_tire 0.4044650571638384
coffee 0.5522071132880543
jump_car 0.5393041337512535
repot 0.519623641296429
cpr 0.5072082759404366
Final MoF: 0.5045616442880024
F1-score
changing_tire 0.3154587889137153
coffee 0.5153851988202668
jump_car 0.5003982406597491
repot 0.4535949823423463
cpr 0.4027056000343274
Final F1-score: 0.437508562154081


In [17]:
seg = SegSlowFast(with_pe=False, features_seg_size=128, cluster=ClusterFactory.get("FINCH")(distance='euclidean'))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.10299665983271168
coffee 0.1883455295637334
jump_car 0.14645566960135742
repot 0.22151038739535
cpr 0.23905351711758513
Final IoU: 0.17967235270214754
MoF
changing_tire 0.35083423622353455
coffee 0.5306230923798089
jump_car 0.5023200919018642
repot 0.5285630713584444
cpr 0.5114775991245001
Final MoF: 0.48476361819763036
F1-score
changing_tire 0.251063780923762
coffee 0.38928572847283327
jump_car 0.4614938690725089
repot 0.42087555562892187
cpr 0.4633239866939544
Final F1-score: 0.3972085841583961


### KMeans + SS + PE

In [25]:
seg = SegSlowFast(with_pe=True, cluster=ClusterFactory.get("KMeans")(n=2))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.26357710499844184
coffee 0.35562147325975735
jump_car 0.3466953772833107
repot 0.33475678157373434
cpr 0.3542196230420969
Final IoU: 0.3309740720314682
MoF
changing_tire 0.3817083649120007
coffee 0.47678624436980344
jump_car 0.4908948315724803
repot 0.4494572350497813
cpr 0.44811282576514677
Final MoF: 0.44939190033384246
F1-score
changing_tire 0.3019728624679467
coffee 0.4086758170622321
jump_car 0.45809987497702476
repot 0.4075571851163389
cpr 0.4090590665180872
Final F1-score: 0.3970729612283259


### KMeans + SS

In [26]:
seg = SegSlowFast(with_pe=False, cluster=ClusterFactory.get("KMeans")(n=2))
seg.segment()
seg.eval_seg.report_results()

IoU
changing_tire 0.2549211894128777
coffee 0.3439061957485229
jump_car 0.33307048793584504
repot 0.32335071674565774
cpr 0.35060418274701405
Final IoU: 0.3211705545179835
MoF
changing_tire 0.3813661055399936
coffee 0.4844258978477935
jump_car 0.4891622286192349
repot 0.4567781139825974
cpr 0.45870825739016036
Final MoF: 0.45408812067595594
F1-score
changing_tire 0.29886162673736844
coffee 0.41249921587217764
jump_car 0.45557546398946047
repot 0.4129989811819878
cpr 0.41478206348191027
Final F1-score: 0.3989434702525809


In [120]:
eval_seg.report_results()

IoU
changing_tire 0.16694321401694148
coffee 0.21584797619909826
jump_car 0.17134433580839462
repot 0.167841989647582
cpr 0.19883358065138215
Final IoU: 0.1841622192646797
MoF
changing_tire 0.11350055985141831
coffee 0.1382858207306055
jump_car 0.1372174406025301
repot 0.14752111141340254
cpr 0.171264555470118
Final MoF: 0.14155789761361487
F1-score
changing_tire 0.12384090734589563
coffee 0.1557366037908081
jump_car 0.1498576305151261
repot 0.15842902444719775
cpr 0.18230455399112247
Final F1-score: 0.15403374401803
